# MultiModal Document RAG with ColQwen2 and Llama 3.2 90B Vision
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/MultiModal_RAG_with_Nvidia_Investor_Slide_Deck.ipynb)

### Install relevant libraries

In [1]:
!pip install byaldi together pdf2image

### Initialize the ColPali Model

In [3]:
import os
from pathlib import Path
from byaldi import RAGMultiModalModel
import torch

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используемое устройство:", device)

Используемое устройство: cuda


In [7]:
# Initialize RAGMultiModalModel
model = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v0.1", device=device)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Lets create our index that will store the embeddings for the page images.

In [9]:
from pathlib import Path

# Указываем локальный путь к папке с PDF-файлами
folder_path = Path("pdfs2")  # Убедитесь, что эта папка находится рядом с вашим .ipynb файлом
index_name = "mts_reports"

metadata_list=[{'data': 'Консолидированная финансовая отчетность 31 декабря 2011 и 2010 годов и за годы, закончившиеся 31 декабря 2011, 2010 и 2009 годов'}, {'data': 'Аудиторское заключение о бухгалтерской отчетности 2010 год'}, {'data': 'Аудиторское заключение о бухгалтерской отчетности 2010 год'}, {'data': 'Аудиторское заключение по бухгалтерской отчетности 2009 год'}, {'data': 'Аудиторское заключение по бухгалтерской отчетности 2011 год'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2024 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2019 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2020 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2021 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2022 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2023 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2019 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2020 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2021 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2022 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 30 июня 2020 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 31 марта 2021 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 31 марта 2022 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 31 марта 2023 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 31 марта 2023 года'}, {'data': 'БУХГАЛТЕРСКИЙ БАЛАНС 31 марта 2024 года'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение независимого аудитора за 2022 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение за 2012 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение за 2013 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение за 2014 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение за 2015 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение за 2016 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение за 2017 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение независимого аудитора за 2018 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение независимого аудитора за 2019 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение независимого аудитора за 2020 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение независимого аудитора за 2021 год'}, {'data': 'Бухгалтерская отчетность и аудиторское заключение независимого аудитора за 2023 год'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора на 31 декабря 2021 и 2020 годов и за 2021, 2020 и 2019 годы'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора а 31 декабря 2022 и 2021 годов и за 2022, 2021 и 2020 годы'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора на 31 декабря 2023 и 2022 годов и за 2023, 2022 и 2021 годы'}, {'data': 'Консолидированная финансовая отчетность на 31 декабря 2015 и 2014 годов и 1 января 2014 года и за годы, закончившиеся 31 декабря 2015 и 2014'}, {'data': 'Консолидированная финансовая отчетность на 31 декабря 2016 и 2015 годов и за годы, закончившиеся 31 декабря 2016, 2015 и 2014 годов'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора за 2017 год'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора за 2018 год'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора за 2019 год'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора на 31 декабря 2013 и 2012 годов и за годы, закончившиеся 31 декабря 2013, 2012 и 2011 годов'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора на 31 декабря 2014 и 2013 годов и за годы, закончившиеся 31 декабря 2014, 2013 и 2012 годов'}, {'data': 'Консолидированная финансовая отчетность и аудиторское заключение независимого аудитора на 31 декабря 2014 и 2013 годов и за годы, закончившиеся 31 декабря 2014, 2013 и 2012 годов'}, {'data': 'Неаудированная сокращенная промежуточная консолидированная финансовая отчетность на 30 июня 2015 года и за шестимесячный период, закончившийся на эту дату'}, {'data': 'Неаудированная сокращенная промежуточная консолидированная финансовая отчетность на 30 июня 2016 года и за шестимесячный период, закончившийся на эту дату'}, {'data': 'Неаудированная сокращенная промежуточная консолидированная финансовая отчетность на 30 июня 2013 года и 31 декабря 2012 года и за шестимесячный период, закончившийся 30 июня 2013 и 2012 годов'}, {'data': 'Неаудированная сокращенная промежуточная консолидированная финансовая отчетность  на 30 июня 2014 года и 31 декабря 2013 года и за шесть месяцев, закончившихся 30 июня 2014 и 2013 годов'}, {'data': 'Сокращенная промежуточная консолидированная финансовая отчетность за шесть месяцев, закончившихся 30 июня 2018 года'}, {'data': 'Сокращенная промежуточная консолидированная финансовая отчетность за шесть месяцев, закончившихся 30 июня 2019 года'}, {'data': 'Сокращенная промежуточная консолидированная финансовая отчетность за шесть месяцев, закончившихся 30 июня 2019 года'}, {'data': 'Сокращенная промежуточная консолидированная финансовая отчетность за шесть месяцев, закончившихся 30 июня 2021 года и 2020 года'}, {'data': 'Сокращенная промежуточная консолидированная финансовая отчетность за шесть месяцев, закончившихся 30 июня 2022 года и 2021 года'}, {'data': 'Сокращенная промежуточная консолидированная финансовая отчетность за шесть месяцев, закончившихся 30 июня 2023 года и 2022 года'}, {'data': 'Сокращенная промежуточная консолидированная финансовая отчетность за шесть месяцев, закончившихся 30 июня 2024 года'}, {'data': 'Неаудированная сокращенная промежуточная консолидированная финансовая отчетность на 30 июня 2012 года и 31 декабря 2011 года и за шесть месяцев, закончившихся 30 июня 2012 и 2011 годов'}],

metadata_dict = {i: metadata for i, metadata in enumerate(metadata_list)}

model.index(
    input_path=folder_path,
    index_name=index_name,
    store_collection_with_index=True,  # Хранит изображения вместе с векторами
    overwrite=True
)



Indexing file: pdfs2\Бухгалтерская+отчетность+ (1).pdf
Added page 1 of document 0 to index.
1730665665.783388
Added page 2 of document 0 to index.
1730665773.4391053
Added page 3 of document 0 to index.
1730665881.2230952
Added page 4 of document 0 to index.
1730665989.1279712
Added page 5 of document 0 to index.
1730666096.6394563
Added page 6 of document 0 to index.
1730666204.2254565
Added page 7 of document 0 to index.
1730666314.341338
Added page 8 of document 0 to index.
1730666422.2115753
Added page 9 of document 0 to index.
1730666530.185712
Added page 10 of document 0 to index.
1730666637.9185817
Added page 11 of document 0 to index.
1730666747.90608
Added page 12 of document 0 to index.
1730666863.113551
Added page 13 of document 0 to index.
1730666977.8228188
Added page 14 of document 0 to index.
1730667087.8317914
Added page 15 of document 0 to index.
1730667196.365081
Added page 16 of document 0 to index.
1730667305.454491
Added page 17 of document 0 to index.
1730667413.9

KeyboardInterrupt: 

## Llama-3.2 11B Vision Model.

In [45]:
query = "Напиши полученные дивиденды за 2009 год"
results = model.search(query, k=5)

print(f"Search results for '{query}':")
for result in results:
    print(f"Doc ID: {result.doc_id}, Page: {result.page_num}, Score: {result.score}")

print("Test completed successfully!")

Search results for 'Напиши полученные дивиденды за 2009 год':
Doc ID: 3, Page: 10, Score: 22.75
Doc ID: 0, Page: 10, Score: 22.25
Doc ID: 3, Page: 35, Score: 22.25
Doc ID: 0, Page: 9, Score: 22.25
Doc ID: 0, Page: 7, Score: 22.125
Test completed successfully!


In [47]:
answer = model.search(query, k=1)
returned_page = answer[0].base64

## Llama-3.2 11B Vision model

In [ ]:
import requests
import base64

url = "https://innoglobalhack-general.olymp.innopolis.university/v1/chat/completions"

# Данные для запроса
data = {
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "add_special_tokens": "true",
    "frequency_penalty": 0.5,
    "temperature": 0.1,
    "max_tokens": 300,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": query
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{returned_page}",
                        "detail": "high"
                    }
                }
            ]
        }
    ]
}

# Отправка POST-запроса
response = requests.post(url, json=data)
# Проверка ответа на успешность запроса
if response.status_code == 200:
    result = response.json()  # Преобразование ответа в JSON
    try:
        # Извлечение текста ответа от модели
        answer = result['choices'][0]['message']['content']
        print(answer)
    except (KeyError, IndexError) as e:
        print("Ошибка при извлечении ответа:", e)
else:
    print(f"Ошибка: {response.status_code} - {response.text}")